In [1]:
from __future__ import division
from __future__ import print_function

import time
import tensorflow as tf

from sklearn import metrics
from utils import *

import random


2024-10-19 16:45:30.060812: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-19 16:45:30.074896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-19 16:45:30.091742: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-19 16:45:30.095217: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-19 16:45:30.108656: I tensorflow/core/platform/cpu_feature_guar

## # The load_corpus function loads the dataset, including the adjacency matrix (adj), node features (features), labels for training, validation, and testing, and their respective masks. These masks are boolean arrays indicating which nodes belong to which subset.


In [2]:
tf.compat.v1.disable_eager_execution()
import os
import random
seed = random.randint(1, 200)
np.random.seed(seed)
# tf.set_random_seed(seed)
tf.compat.v1.set_random_seed(seed)
os.environ["CUDA_VISIBLE_DEVICES"] = ""


dataset = '1500'  # 'citeseer' or 'pubmed' as alternatives
model_type = 'GCN'  # 'gcn_cheby' or 'dense' as alternatives
learning_rate = 0.02
epochs = 200
hidden1 = 200
dropout = 0.5
weight_decay = 0  # 5e-4 can be used as well
early_stopping = 10
max_degree = 3
# import argparse#########

# parser = argparse.ArgumentParser()##########
# FLAGS = parser.parse_args()############
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus(dataset)
print("Adj:\n",adj)
print("Features:\n",features)
print("Y_train:\n",y_train)
print("Y_val:\n",y_val)
print("Y_test:\n",y_test)
print("Train_mask:\n",train_mask)
print("Val_mask:\n",val_mask)
print("Test_mask:\n",test_mask)
print("Train_size:\n",train_size)
print("Test_size:\n",test_size)

(944, 300) (944, 38) (228, 300) (228, 38) (5780, 300) (5780, 38)
6008
Adj:
 <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1667912 stored elements and shape (6008, 6008)>
  Coords	Values
  (0, 1090)	2.7447662166404823
  (0, 1145)	2.640625957387885
  (0, 1158)	14.862782659705548
  (0, 1256)	4.207046484738295
  (0, 1271)	2.87481934488868
  (0, 1308)	2.6081906816347313
  (0, 1403)	3.9734316335567894
  (0, 1487)	4.848900370910689
  (0, 1488)	1.8236092951151541
  (0, 1535)	3.784189633918261
  (0, 1563)	3.367295829986474
  (0, 1623)	3.033391721470971
  (0, 1684)	3.2196598311804094
  (0, 1691)	3.984860329380412
  (0, 1695)	4.261113706008571
  (0, 1769)	3.0571409016826343
  (0, 1824)	4.7535901911063645
  (0, 1884)	3.9326096390365346
  (0, 1931)	2.472893019285368
  (0, 1943)	4.060443010546419
  (0, 2127)	2.227229087965195
  (0, 2195)	1.8333654700605189
  (0, 2220)	4.060443010546419
  (0, 2245)	3.893388925883253
  (0, 2298)	2.536364947063476
  :	:
  (6007, 4514)	3.625124939288574


In [3]:
features = sp.identity(features.shape[0])  # featureless

print(adj.shape)
print(features.shape)

# Some preprocessing
features = preprocess_features(features)

(6008, 6008)
(6008, 6008)


In [4]:
from models import GCN

usage: ipykernel_launcher.py [-h]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/sjain/.local/share/jupyter/runtime/kernel-aa1904e1-e079-472d-ae75-9854b4501071.json


SystemExit: 2

/home/sjain/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:

# # Set random seed
# seed = random.randint(1, 200)
# np.random.seed(seed)
# tf.set_random_seed(seed)

# # Settings
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

# flags = tf.app.flags
# FLAGS = flags.FLAGS
# # 'cora', 'citeseer', 'pubmed'
# flags.DEFINE_string('dataset', dataset, 'Dataset string.')
# # 'gcn', 'gcn_cheby', 'dense'
# flags.DEFINE_string('model', 'gcn', 'Model string.')
# flags.DEFINE_float('learning_rate', 0.02, 'Initial learning rate.')
# flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
# flags.DEFINE_integer('hidden1', 200, 'Number of units in hidden layer 1.')
# flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
# flags.DEFINE_float('weight_decay', 0,
#                    'Weight for L2 loss on embedding matrix.')  # 5e-4
# flags.DEFINE_integer('early_stopping', 10,
#                      'Tolerance for early stopping (# of epochs).')
# flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

# # Load data
# adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask, train_size, test_size = load_corpus(
#     FLAGS.dataset)
# print(adj)
# print(adj[0], adj[1])

if FLAGS.model == 'gcn':
    support = [preprocess_adj(adj)]
    num_supports = 1
    model_func = GCN
###########################commented by us#
# elif FLAGS.model == 'gcn_cheby':
#     support = chebyshev_polynomials(adj, FLAGS.max_degree)
#     num_supports = 1 + FLAGS.max_degree
#     model_func = GCN
# elif FLAGS.model == 'dense':
#     support = [preprocess_adj(adj)]  # Not used
#     num_supports = 1
#     model_func = MLP
# else:
#     raise ValueError('Invalid argument for model: ' + str(FLAGS.model))

# Define placeholders

#TensorFlow Placeholders: Placeholders are created for feeding data into the model. This includes the support matrices (for GCN), input features, labels, and dropout rates.
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    # helper variable for sparse dropout
    'num_features_nonzero': tf.placeholder(tf.int32)
}

# Create model
print(features[2][1])
model = model_func(placeholders, input_dim=features[2][1], logging=True)

# Initialize session
session_conf = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=session_conf)


# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(
        features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy, model.pred, model.labels], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], outs_val[2], outs_val[3], (time.time() - t_test)


# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(
        features, support, y_train, train_mask, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy,
                     model.layers[0].embedding], feed_dict=feed_dict)

    # Validation
    cost, acc, pred, labels, duration = evaluate(
        features, support, y_val, val_mask, placeholders)
    cost_val.append(cost)

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(
              outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

# Testing
test_cost, test_acc, pred, labels, test_duration = evaluate(
    features, support, y_test, test_mask, placeholders)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))

test_pred = []
test_labels = []
print(len(test_mask))
for i in range(len(test_mask)):
    if test_mask[i]:
        test_pred.append(pred[i])
        test_labels.append(labels[i])

print("Test Precision, Recall and F1-Score...")
print(metrics.classification_report(test_labels, test_pred, digits=4))
print("Macro average Test Precision, Recall and F1-Score...")
print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='macro'))
print("Micro average Test Precision, Recall and F1-Score...")
print(metrics.precision_recall_fscore_support(test_labels, test_pred, average='micro'))


NameError: name 'GCN' is not defined

In [ ]:

# doc and word embeddings
print('embeddings:')
word_embeddings = outs[3][train_size: adj.shape[0] - test_size]
train_doc_embeddings = outs[3][:train_size]  # include val docs
test_doc_embeddings = outs[3][adj.shape[0] - test_size:]

print(len(word_embeddings), len(train_doc_embeddings),
      len(test_doc_embeddings))
print(word_embeddings)

f = open('data/corpus/_vocab.txt', 'r')
words = f.readlines()
f.close()

vocab_size = len(words)
word_vectors = []
for i in range(vocab_size):
    word = words[i].strip()
    word_vector = word_embeddings[i]
    word_vector_str = ' '.join([str(x) for x in word_vector])
    word_vectors.append(word + ' ' + word_vector_str)

word_embeddings_str = '\n'.join(word_vectors)
f = open('data/_word_vectors.txt', 'w')
f.write(word_embeddings_str)
f.close()

doc_vectors = []
doc_id = 0
for i in range(train_size):
    doc_vector = train_doc_embeddings[i]
    doc_vector_str = ' '.join([str(x) for x in doc_vector])
    doc_vectors.append('doc_' + str(doc_id) + ' ' + doc_vector_str)
    doc_id += 1

for i in range(test_size):
    doc_vector = test_doc_embeddings[i]
    doc_vector_str = ' '.join([str(x) for x in doc_vector])
    doc_vectors.append('doc_' + str(doc_id) + ' ' + doc_vector_str)
    doc_id += 1

doc_embeddings_str = '\n'.join(doc_vectors)
f = open('data/_doc_vectors.txt', 'w')
f.write(doc_embeddings_str)
f.close()